In [2]:
!pwd

/home/mln/GIT/0_EXPLORE/github/astronomer-codespaces-test/data


In [7]:
import pandas as pd


df = pd.read_parquet('2024-07-24.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942480 entries, 0 to 942479
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype                                
---  ------             --------------   -----                                
 0   id                 942480 non-null  int64                                
 1   record_timestamp   942480 non-null  datetime64[us, pytz.FixedOffset(120)]
 2   stationcode        942480 non-null  object                               
 3   ebike              942480 non-null  int64                                
 4   mechanical         942480 non-null  int64                                
 5   duedate            942480 non-null  object                               
 6   numbikesavailable  942480 non-null  int64                                
 7   numdocksavailable  942480 non-null  int64                                
 8   capacity           942480 non-null  int64                                
 9   is_renting     

In [10]:
import pandas as pd


df = pd.read_parquet('2024-06-04.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863464 entries, 0 to 863463
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype                                
---  ------                       --------------   -----                                
 0   id                           0 non-null       float64                              
 1   record_timestamp             863464 non-null  datetime64[us, pytz.FixedOffset(120)]
 2   stationcode                  863464 non-null  object                               
 3   ebike                        863464 non-null  int64                                
 4   mechanical                   863464 non-null  int64                                
 5   duedate                      863464 non-null  object                               
 6   numbikesavailable            863464 non-null  int64                                
 7   numdocksavailable            863464 non-null  int64                                

In [5]:
import pandas as pd



df = pd.read_parquet('stations_migration.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46581312 entries, 0 to 46581311
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id                 float64
 1   record_timestamp   object 
 2   stationcode        object 
 3   ebike              int64  
 4   mechanical         int64  
 5   duedate            object 
 6   numbikesavailable  int64  
 7   numdocksavailable  int64  
 8   capacity           int64  
 9   is_renting         object 
 10  is_installed       object 
 11  is_returning       object 
dtypes: float64(1), int64(5), object(6)
memory usage: 4.2+ GB


In [8]:
import pandas as pd



df = pd.read_parquet('locations_backup.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   stationcode                  1496 non-null   object 
 1   name                         1496 non-null   object 
 2   latitude                     1496 non-null   float64
 3   longitude                    1496 non-null   float64
 4   nom_arrondissement_communes  1496 non-null   object 
dtypes: float64(2), object(3)
memory usage: 58.6+ KB
